In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [10]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [11]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_testing_noanem_noisy_6.csv')
# df = df.fillna(-1)
# df.head()
train_df = pd.read_csv('../../final/data/train_set_missing_2.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,10.706860,-1.000000,-1.000000,-1.000000,308.295738,104.212335,50.527083,3.082224,1,0.410824,65.084260,118.607783,45.819372,21.282930,50.512056,32.120580,16.389161,1
1,8.502071,-1.000000,0.106615,6.434651,-1.000000,96.250534,-1.000000,2.649981,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,25.506212,-1.000000,6
2,17.134148,450.994047,2.785983,3.580950,221.642272,95.404759,-1.000000,5.387828,1,1.004988,51.333192,-1.000000,-1.000000,-1.000000,-1.000000,51.402444,-1.000000,0
3,12.600706,307.881516,-1.000000,-1.000000,-1.000000,-1.000000,68.163818,4.127231,1,-1.000000,8.894795,-1.000000,26.676604,2.402901,73.780752,37.802117,-1.000000,5
4,11.708159,-1.000000,5.601042,-1.000000,-1.000000,92.100873,46.823176,-1.000000,1,-1.000000,-1.000000,54.558476,-1.000000,25.629368,-1.000000,-1.000000,-1.000000,5


In [12]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,4.082935,0,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,35.349018,-1.000000,7
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,5.038345,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000,3
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,3.165345,1,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,23.820517,-1.000000,4
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,2.852876,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.000000,5
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,4.848900,1,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,37.079144,10.501658,4


In [13]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [14]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

#### Training 

In [ ]:
for steps in [int(5e6), int(5.5e6), int(6e6), int(6.5e6), int(7e6), int(7.5e6), int(8e6), int(8.5e6), int(9e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_missing_2_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.81     |
|    ep_rew_mean      | -0.92    |
|    exploration_rate | 0.421    |
|    success_rate     | 0.08     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 913      |
|    time_elapsed     | 333      |
|    total_timesteps  | 304952   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.36     |
|    n_updates        | 63737    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.39     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.07     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 756      |
|    t

In [ ]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])